# Face Recognition Using CNN Architecture in Python

## About the Dataset (Images)

> The data contains cropped face images of 16 people divided into Training and testing. We will train the CNN model using the images in the Training folder and then test the model by using the unseen images from the testing folder, to check if the model is able to recognise the face number i.e label given to each face during training of the unseen images or not.

In [1]:
# import libraries 
import tensorflow
from tensorflow import keras

# Image Agumentation with Preprocessing

In [2]:
train_images = r"Face Images/Face Images/Final Training Images"

## Image Preprocessing using keras


# As we know deep-learning is hungry for data, the data we have is only limited. 
# so lets perform **Image Agumentation** to create different versions
# of the original image, which leads to a better model, since it learns
# on the good and bad mix of images.

from keras.preprocessing.image import ImageDataGenerator
train_gen = ImageDataGenerator(
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True
)
# No transformations are made on the test data
test_gen = ImageDataGenerator()

# Generating training data
training_data = train_gen.flow_from_directory(
    train_images, 
    target_size = (100,100),
    batch_size = 30,
    class_mode = 'categorical'
)

# generating test data
testing_data = test_gen.flow_from_directory(
    train_images, 
    target_size = (100,100),
    batch_size = 30,
    class_mode = 'categorical'
)

# Printing class labels for each face
testing_data.class_indices

Found 244 images belonging to 16 classes.
Found 244 images belonging to 16 classes.


{'face1': 0,
 'face10': 1,
 'face11': 2,
 'face12': 3,
 'face13': 4,
 'face14': 5,
 'face15': 6,
 'face16': 7,
 'face2': 8,
 'face3': 9,
 'face4': 10,
 'face5': 11,
 'face6': 12,
 'face7': 13,
 'face8': 14,
 'face9': 15}

## Mapping of class_labels with numeric value for classification

In [4]:
# training_data have numeric tag for each face
Train_class = training_data.class_indices

# lets store them in a dictionary with swap for future reference
Result_class = {}
for value_tag, face_tag in zip(Train_class.values(),Train_class.keys()):
    Result_class[value_tag] = face_tag

    
# use pickle to save the mapping's
import pickle
with open(r'Face Images/Face Images/ResultMap.pkl','wb') as Final_mapping:
    pickle.dump(Result_class,Final_mapping)

In [5]:
print("Mapping of Face and its numeric value",Result_class)

Mapping of Face and its numeric value {0: 'face1', 1: 'face10', 2: 'face11', 3: 'face12', 4: 'face13', 5: 'face14', 6: 'face15', 7: 'face16', 8: 'face2', 9: 'face3', 10: 'face4', 11: 'face5', 12: 'face6', 13: 'face7', 14: 'face8', 15: 'face9'}


In [6]:
Output_Neurons=len(Result_class)
print('\n The Number of output neurons: ', Output_Neurons)


 The Number of output neurons:  16


# Building the CNN Architecture, Model Compilation and Training

**In the below code snippet, I have created a CNN model with**

3. hidden layers of convolution
3. hidden layers of max pooling
1. layer of flattening
2. Hidden ANN layer
1. output layer with 16-neurons (one for each face)

In [8]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense

In [14]:
import numpy as np
import tensorflow as tf
from keras.callbacks import Callback, EarlyStopping
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense
import time

# Definisikan callback kustom untuk menghentikan pelatihan ketika akurasi > 96%
class CustomEarlyStopping(Callback):
    def on_epoch_end(self, epoch, logs=None):
        if logs.get('val_accuracy') >= 0.96:
            print(f"\nAkurasi validasi mencapai {logs.get('val_accuracy') * 100:.2f}%, menghentikan pelatihan!")
            self.model.stop_training = True

# Definisikan model
Model = Sequential()
Model.add(Conv2D(16, kernel_size=(5,5), strides=(1,1), input_shape=(100, 100, 3), activation='relu'))
Model.add(MaxPool2D(pool_size=(2,2)))
Model.add(Conv2D(32, kernel_size=(3,3), strides=(1,1), activation='relu'))
Model.add(MaxPool2D(pool_size=(2,2)))
Model.add(Flatten())
Model.add(Dense(64, activation='relu'))
Model.add(Dense(Output_Neurons, activation='softmax'))

# Kompilasi model
Model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Menggunakan Early stopping untuk mengurangi waktu pelatihan
call = CustomEarlyStopping()

# Mengukur waktu yang dibutuhkan oleh model untuk melatih
StartTime = time.time()

# Melatih model
Model.fit(training_data,
          epochs=30,
          validation_data=testing_data,
          callbacks=[call])

Endtime = time.time()
print('Total Training Time taken: ', round((Endtime - StartTime) / 60), 'Minutes')


Epoch 1/30
9/9 [==============================] - 3s 245ms/step - loss: 238.5195 - accuracy: 0.0697 - val_loss: 48.7268 - val_accuracy: 0.1230
Epoch 2/30
9/9 [==============================] - 2s 204ms/step - loss: 18.2782 - accuracy: 0.1680 - val_loss: 3.3543 - val_accuracy: 0.3566
Epoch 3/30
9/9 [==============================] - 2s 207ms/step - loss: 2.5552 - accuracy: 0.3320 - val_loss: 2.1222 - val_accuracy: 0.3934
Epoch 4/30
9/9 [==============================] - 2s 207ms/step - loss: 2.0935 - accuracy: 0.3607 - val_loss: 1.8493 - val_accuracy: 0.4754
Epoch 5/30
9/9 [==============================] - 2s 263ms/step - loss: 1.9565 - accuracy: 0.4057 - val_loss: 1.5400 - val_accuracy: 0.5656
Epoch 6/30
9/9 [==============================] - 2s 249ms/step - loss: 1.5350 - accuracy: 0.5615 - val_loss: 1.1531 - val_accuracy: 0.6475
Epoch 7/30
9/9 [==============================] - 2s 244ms/step - loss: 1.1252 - accuracy: 0.6844 - val_loss: 0.8388 - val_accuracy: 0.7951
Epoch 8/30
9/9 [

In [18]:
Model.save('face_recognition_model.h5')

C:\Users\ACER\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


## Using the Classifier to make predictions on unseen test images

In [15]:
import numpy as np
from keras.preprocessing import image
import os
import glob

In [16]:
'''########### Making single predictions ###########'''

ImagePath=r"Face Images/Face Images/Final Training Images/face1/image_0001_Face_1.jpg"
test_image=image.load_img(ImagePath,target_size=(100, 100))
test_image=image.img_to_array(test_image)
 
test_image=np.expand_dims(test_image,axis=0)
 
result=Model.predict(test_image,verbose=0)
#print(training_set.class_indices)
 
print('####'*10)
print('Prediction is: ',Result_class[np.argmax(result)])


########################################
Prediction is:  face1


In [17]:
'''############ Making multiple predictions ###########'''

## Loading all the image paths from final testing folder for prediction
main_ = r"Face Images/Face Images/Final Testing Images"
img_paths = glob.glob(os.path.join(main_,'**','*.jpg'))

print(img_paths[0:5]) # every image will be a PIL object
print('*'*50)

for path in img_paths:
    test_image = image.load_img(path,target_size=(100,100))
    test_image = image.img_to_array(test_image)
    test_image = np.expand_dims(test_image,axis =0)
    result = Model.predict(test_image,verbose=0)
    print('Prediction: ',Result_class[np.argmax(result)])

['Face Images/Face Images/Final Testing Images\\face1\\1face1.jpg', 'Face Images/Face Images/Final Testing Images\\face1\\2face1.jpg', 'Face Images/Face Images/Final Testing Images\\face1\\3face1.jpg', 'Face Images/Face Images/Final Testing Images\\face1\\4face1.jpg', 'Face Images/Face Images/Final Testing Images\\face10\\1face10.jpg']
**************************************************
Prediction:  face1
Prediction:  face1
Prediction:  face13
Prediction:  face4
Prediction:  face10
Prediction:  face10
Prediction:  face10
Prediction:  face10
Prediction:  face11
Prediction:  face11
Prediction:  face11
Prediction:  face11
Prediction:  face12
Prediction:  face12
Prediction:  face12
Prediction:  face12
Prediction:  face4
Prediction:  face13
Prediction:  face13
Prediction:  face16
Prediction:  face14
Prediction:  face14
Prediction:  face14
Prediction:  face14
Prediction:  face15
Prediction:  face15
Prediction:  face15
Prediction:  face15
Prediction:  face16
Prediction:  face16
Prediction:  fa